In [6]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [7]:
import minsearch
import json
import os
from dotenv import load_dotenv
import openai

In [8]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [9]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [10]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course']
)

In [11]:
index.fit(documents)
q = 'the course has started, can i still enroll?'
boost = {'question':3.0, 'section': 0.5}

In [12]:
results = index.search(
    query=q,
    boost_dict=boost,
    num_results=5,
    filter_dict={'course': 'data-engineering-zoomcamp'}
)

In [13]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [14]:
from openai import OpenAI
load_dotenv()

True

In [15]:
client = OpenAI()

In [16]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"It's possible to enroll in a course after it has started, but it depends on the institution's policies and the specific course in question. Some courses may allow late enrollment with certain conditions, such as catching up on missed material or obtaining special permission from the instructor. I recommend contacting the course instructor or the institution's registrar office as soon as possible to inquire about your options for late enrollment. They will provide you with the most accurate information regarding deadlines and any additional steps you may need to take. \n\nIs this an online course or an in-person class? Different formats may have different policies as well."

In [17]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

context = ""
    
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

prompt = prompt_template.format(question=q, context=context).strip()

In [18]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

'Yes, you can still enroll in the course even after it has started. You are eligible to submit the homework, but be mindful of the deadlines for the final projects. It is important not to leave everything for the last minute.'

In [19]:
# Clean code in functions:

In [20]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [21]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [22]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [23]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [24]:
rag(query)

'To run Kafka in the terminal, follow these instructions:\n\nFor running a Java Kafka producer/consumer using a JAR file, in the project directory, execute the following command:\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nMake sure to replace `<jar_name>` with the actual name of your JAR file.\n\nIf you are working with Python and need to run a Kafka producer, you should create and activate a virtual environment, install the necessary packages, and then run your Python script. Here’s how you can do that:\n\n1. Create and activate a virtual environment:\n    ```bash\n    python -m venv env\n    source env/bin/activate  # On Windows, use `env\\Scripts\\activate`\n    ```\n\n2. Install the required packages:\n    ```bash\n    pip install -r ../requirements.txt\n    ```\n\n3. Run your Python script within the virtual environment. For example:\n    ```bash\n    python producer.py\n    ```\n\nMake sure all your Docker ima

In [25]:
rag('the course has already started, can I still enroll?')

'Yes, you can still enroll in the course even after it has started. You are also eligible to submit the homeworks. However, be mindful of the deadlines for turning in the final projects and avoid leaving everything until the last minute.'

In [26]:
from elasticsearch import Elasticsearch

In [27]:
es_client = Elasticsearch('http://localhost:9200') 

In [29]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [33]:
from tqdm.auto import tqdm

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  1%|          | 8/948 [00:00<00:25, 36.52it/s]

100%|██████████| 948/948 [00:21<00:00, 43.13it/s]


In [35]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [36]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [37]:
rag('the course has already started, can I still enroll?')

"Yes, the course allows late enrollment. Even if you didn't register before the course started, you can still submit the homework assignments. However, be mindful of the deadlines for submitting final projects and avoid procrastinating."